# Introduction to the AlgoSeek SDK

## 1) Overview

In [1]:
# import libraries
import algoseek_connector
import algoseek_connector.functions as fn
import pandas as pd
import os
from dotenv import load_dotenv

pd.set_option('display.max_columns',150)
pd.set_option('display.max_rows', 100)

load_dotenv()

jhost = os.getenv('host_julian')
juser = os.getenv('user_julian')
jpass = os.getenv('password_julian')

session = algoseek_connector.Session(
    jhost, juser,jpass
)
session.ping() # Verify that the session was instantiated corrected
resource = algoseek_connector.DataResource(session)

## 2) US Equity Reference Data

In [ ]:
for dataset in resource.datagroups.USEquityReferenceData.all():
    print(dataset)

### 2.1) Basic Adjustment Factors

In [10]:
basic_adj = resource.datagroups.USEquityReferenceData.datasets.BasicAdjustments
basic_adj.head().fetch().columns

Index(['SecId', 'Ticker', 'EffectiveDate', 'AdjustmentFactor',
       'AdjustmentReason', 'EventId'],
      dtype='object')

Basic Adjustment Factor Data Fields:

| Field            | Type (Format) | Description                                                                                      |
|------------------|---------------|--------------------------------------------------------------------------------------------------|
| SecId            | integer | Unique Security Identifier                                                                       |
| Ticker           | string | Ticker on the Effective Date                                                                     |
| EffectiveDate    | string (YYYYMMDD) | Date that the event becomes effective                                                            |
| AdjustmentFactor | decimal | The value of the adjustment factor for the event |
| AdjustmentReason | string | The reason for the Corporate Event. See subsection “Adjustment Reason” below for a list of types |
| EventId | integer | Unique Event ID under the AdjustmentReason, i.e., each pair of AdjustmentReason and EventId is unique |

In [16]:
basic_df = basic_adj.select(basic_adj.SecId,
                 basic_adj.Ticker,
                 basic_adj.EffectiveDate,
                 basic_adj.AdjustmentFactor,
                 basic_adj.AdjustmentReason,
                 basic_adj.EventId
).fetch()
basic_df.to_parquet('data/us_equity/reference/basic_adj.parquet')

### 3.2) Detailed Adjustment Factors

In [17]:
detailed_adj = resource.datagroups.USEquityReferenceData.datasets.DetailedAdjustments
detailed_adj.head().fetch().columns

Index(['SecId', 'Ticker', 'Name', 'ISIN', 'EffectiveDate', 'ReportDate',
       'Revision', 'AdjustmentFactor', 'AdjustmentReason', 'EventType',
       'EventId', 'DivPayrate', 'DivCurrency', 'DivPaymentType', 'Detail'],
      dtype='object')

Detailed Adjustment Factor data fields:

| Field            | Type (Format)               | Description                                                                                                                   |
|------------------|-----------------------------|-------------------------------------------------------------------------------------------------------------------------------|
| SecId            | integer                     | Unique security identifier                                                                                                    |
| Ticker           | string                      | Ticker on the Effective Date                                                                                                  |
| EffectiveDate    | string (YYYYMMDD)           | Date that the event becomes effective                                                                                         |
| Name             | string | Name of the company                                                                                                           |
| ReportDate       | string (YYYYMMDD[_N]        | Date and version (_N suffix) of the algoseek event report file                                                                |
| AdjustmentFactor | decimal                     | The value of the Adjustment factor for the event                                                                              |
| AdjustmentReason | string                      | The reason for the Corporate Event. See subsection "Adjustment Reason" below for a list of types                              |
| EventType        | string                      | Type of event. See section “Adjustment Reason Table” below for a list of types                                                |
| EventId          | integer                     | Unique Event ID under the AdjustmentReason, i.e., each pair of AdjustmentReason and EventId is unique                         |
| Detail           | string                      | Details of the event including Dividend Amount, Period (e.g., Interim), etc. See “Detail Field” below for further information |
| ISIN             | string                      | ISIN (International SecurNumber) Identifier|

In [18]:
det_df = detailed_adj.select(detailed_adj.SecId,
                    detailed_adj.Ticker,
                    detailed_adj.Name,
                    detailed_adj.ISIN,
                    detailed_adj.EffectiveDate,
                    detailed_adj.ReportDate,
                    detailed_adj.Revision,
                    detailed_adj.AdjustmentFactor,
                    detailed_adj.AdjustmentReason,
                    detailed_adj.EventType,
                    detailed_adj.EventId,
                    detailed_adj.DivPayrate,
                    detailed_adj.DivCurrency,
                    detailed_adj.DivPaymentType,
                    detailed_adj.Detail
                    ).fetch()
det_df.to_parquet('data/us_equity/reference/detailed_adj.parquet')

### 2.3) Lookup Base

In [11]:
lookup_base = resource.datagroups.USEquityReferenceData.datasets.LookupBase
lookup_base.head().fetch().columns

Index(['SecId', 'Ticker', 'StartDate', 'EndDate'], dtype='object')

Lookup Base Fields

In [19]:
look_df = lookup_base.select(lookup_base.SecId,
                   lookup_base.Ticker,
                   lookup_base.StartDate,
                   lookup_base.EndDate
).fetch()
look_df.to_parquet('data/us_equity/reference/lookup_base.parquet')

### 2.4) SEC Master Base

In [12]:
sec_master_base = resource.datagroups.USEquityReferenceData.datasets.SecMasterBase
sec_master_base.head().fetch().columns

Index(['SecId', 'ListStatus', 'SecurityDescription', 'Sic', 'Sector',
       'Industry', 'SEDOL', 'Ticker', 'TickerStartToEndDate', 'Name',
       'NameStartToEndDate', 'ISIN', 'ISINStartToEndDate', 'USIdentifier',
       'USIdentifierStartToEndDate', 'PrimaryExchange',
       'PrimaryExchangeStartToEndDate', 'FIGI'],
      dtype='object')

SEC Master Base Datafields

In [ ]:
sec_df = sec_master_base.select(sec_master_base.SecId,
                       sec_master_base.ListStatus,
                       sec_master_base.SecurityDescription,
                       sec_master_base.Sic,
                       sec_master_base.Sector,
                       sec_master_base.Industry,
                       sec_master_base.SEDOL,
                       sec_master_base.Ticker,
                       sec_master_base.TickerStartToEndDate,
                       sec_master_base.Name,
                       sec_master_base.NameStartToEndDate,
                       sec_master_base.ISIN,
                       sec_master_base.ISINStartToEndDate,
                       sec_master_base.USIdentifier,
                       sec_master_base.USIdentifierStartToEndDate,
                       sec_master_base.PrimaryExchange,
                       sec_master_base.PrimaryExchangeStartToEndDate,
                       sec_master_base.FIGI
).fetch()
sec_df.to_parquet('data/us_equity/reference/sec_master.parquet')

## 3) US Equity Market Data

These cells deal with downloading market data for US Equities. Run whichever cells download the data that you want then skip the rest. Make sure you go to the end to download the reference data.

In [5]:
# List all available datasets
for dataset in resource.datagroups.USEquityMarketData.datasets.all():
    print(dataset.name)

BasicAdjustedOHLCDaily
BasicOHLCDaily
PrimaryAdjustedOHLCDaily
PrimaryOHLCDaily
StandardAdjustedOHLCDaily
StandardOHLCDaily
TradeAndQuote
TradeAndQuoteMinuteBar
TradeAndQuoteMinuteBarExcludingTRF
TradeOnly
TradeOnlyAdjusted
TradeOnlyAdjustedMinuteBar
TradeOnlyAdjustedMinuteBarBBG
TradeOnlyAdjustedMinuteBarExcludingTRF
TradeOnlyMinuteBar
TradeOnlyMinuteBarBBG
TradeOnlyMinuteBarExcludingTRF


### 3.1) Basic OHLC Daily

### 3.2) Basic Adjusted OHLC Daily

### 3.3) Primary OHLC Daily

In [6]:
look = pd.read_parquet('data/us_equity/ref/lookup.parquet')
tickers = look['Ticker'].unique()

In [8]:
tickers

array(['SLF', 'ASTSF', 'PGLA', ..., 'IILGV', 'WRKw', 'RACEw'],
      dtype=object)

array(['SLF', 'ASTSF', 'PGLA', ..., 'IILGV', 'WRKw', 'RACEw'],
      dtype=object)

In [ ]:
from tqdm import tqdm
primary_ohlc_daily = resource.datagroups.USEquityMarketData.datasets.PrimaryOHLCDaily

for tick in tqdm(tickers):
    print(tick)

    tic = primary_ohlc_daily.select(
           primary_ohlc_daily.TradeDate,
           primary_ohlc_daily.SecId,
           primary_ohlc_daily.Ticker,
           primary_ohlc_daily.Name,
           primary_ohlc_daily.PrimaryExchange,
           primary_ohlc_daily.ISIN,
           primary_ohlc_daily.OpenTime,
           primary_ohlc_daily.OpenPrice,
           primary_ohlc_daily.OpenSize,
           primary_ohlc_daily.HighTime,
           primary_ohlc_daily.HighPrice,
           primary_ohlc_daily.LowTime,
           primary_ohlc_daily.LowPrice,
           primary_ohlc_daily.CloseTime,
           primary_ohlc_daily.ClosePrice,
           primary_ohlc_daily.CloseSize,
           primary_ohlc_daily.ListedMarketHoursVolume,
           primary_ohlc_daily.ListedMarketHoursTrades,
           primary_ohlc_daily.ListedTotalVolume,
           primary_ohlc_daily.ListedTotalTrades,
           primary_ohlc_daily.FinraMarketHoursVolume,
           primary_ohlc_daily.FinraMarketHoursTrades,
           primary_ohlc_daily.FinraTotalVolume,
           primary_ohlc_daily.FinraTotalTrades,
           primary_ohlc_daily.MarketVWAP,
           primary_ohlc_daily.DailyVWAP
    ).filter(
        (primary_ohlc_daily.Ticker==tick)
    ).fetch()
    tic.to_parquet('data/us_equity/primary_daily/{stock}.parquet'.format(stock=tick))

  0%|                                                                     | 0/24745 [00:00<?, ?it/s]

SLF


  0%|                                                           | 1/24745 [00:01<7:48:30,  1.14s/it]

ASTSF


  0%|                                                           | 2/24745 [00:01<5:23:02,  1.28it/s]

...

 26%|███████████████▏                                          | 6490/24745 [22:15<56:54,  5.35it/s]

PEpB
BOTJ


 26%|██████████████▋                                         | 6491/24745 [22:15<1:01:19,  4.96it/s]

JDO


 26%|██████████████▋                                         | 6492/24745 [22:15<1:01:53,  4.92it/s]

### 3.4) Primary Adjusted OHLC Daily

### 3.5) Standard OHLC Daily

### 3.6) Standard Adjusted OHLC Daily

### 3.7) Trade and Quote

### 3.8) Trade and Quote Minute Bar

### 3.9) Trade and Quote Minute Bar Excluding TRF

### 3.10) Trade Only

### 3.11) Trade Only Adjusted

### 3.12) Trade Only Adjusted Minute Bar

### 3.13) Trade Only Adjusted Minute Bar BBG

### 3.14) Trade Only Adjusted Minute Bar Excluding TRF

### 3.15) Trade Only Minute Bar

### 3.16) Trade Only Minute Bar BBG

### 3.17) Trade Only Minute Bar Excluding TRF